In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
import statistics as stats
import math
#import statsmodels.formula.api as sm

In [2]:
original_df = pd.read_csv('newyork_housing.csv')

/tmp/ipykernel_3453282/1156967300.py:1: DtypeWarning: Columns (2,5,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,305,313,32

In [3]:
df = original_df.copy()
#print(df, df.shape)

In [4]:
#Schools: changes done
#Notes:
#1. "assigned" is 99% = blank (dropped)
#2. "isAssigned" is 99% = 1 (dropped)
#3. "totalCount" is really uninterpretable and we can use "#of schools listed by level" as a substitute (dropped)
#4. "grades" in [chartered, public] is meaningless (dropped)
#5. "size" can be confounding to "spt" (dropped)
#6. "rating" can be confounding due to different metrics accross levels of school. (dropped)
school = df.filter(regex = 'school')
for col in school.columns:
    for w in ['link', 'name','size','assigned','totalCount','grades','rating','type','isAssigned']:
        if w in col:
            school = school.drop(columns = [col],axis=1)
school = school.drop(columns = ['schools']) #blank col
print(school.describe())
print(school.isnull().sum())
print(school)
school = school.replace('Primary','Elementary')

       schools/0/distance  schools/0/studentsPerTeacher  schools/1/distance  \
count        75499.000000                  74504.000000        74719.000000   
mean             0.389759                     13.921185            0.900305   
std              0.839690                      2.048344            1.087598   
min              0.000000                      6.000000            0.000000   
25%              0.200000                     13.000000            0.400000   
50%              0.300000                     14.000000            0.700000   
75%              0.500000                     15.000000            1.200000   
max            213.100000                     20.000000          214.800000   

       schools/1/studentsPerTeacher  schools/2/distance  \
count                  72815.000000        55543.000000   
mean                      14.312230            1.166651   
std                        2.423886            1.244332   
min                        7.000000            0.000

The following to blacks are just to show there are properties that have
1. multiple schools at same level around
2. duplicate of schools
1&2 implies averaging by level might be the best solution. 

In [5]:
#school['schools/0/elem']=(school['schools/0/level']==school['schools/1/level'])&(school['schools/0/distance']==school['schools/1/distance'])&(school['schools/0/studentsPerTeacher']==school['schools/1/studentsPerTeacher']).astype(int)

In [6]:
#print(school['schools/0/elem'].sum())
#print(school.loc[school['schools/0/elem'] == 1])

Can ignore the following
What I was doing is calculate the total number of schools around by school level.
No longer need it here.

In [7]:
#print(school.columns)
#for col in school.columns:
#    for i in ['s1','s2','s3']:
#        if i in col:
#            school = school.drop([col],axis=1)
#print(school)

In [8]:
#print(school.columns)
#print(school[['school/elem/num','school/midd/num','school/high/num']].describe())

In [9]:
#Main code
#generate dummies each "schools/*/level" => s1_*, s2_*, s3_*
school = school.join(pd.get_dummies(school['schools/0/level'],prefix='s1'))
school = school.join(pd.get_dummies(school['schools/1/level'],prefix='s2'))
school = school.join(pd.get_dummies(school['schools/2/level'],prefix='s3'))
print(school)
#sum across levels
#i.e. 'school/elem/num' = total number of elementary schools listed
school['school/elem/num'] = school['s1_Elementary'] + school['s2_Elementary'] + school['s3_Elementary']
school['school/midd/num'] = school['s1_Middle'] + school['s2_Middle'] + school['s3_Middle']
school['school/high/num'] = school['s2_High'] + school['s3_High'] #note: "schools/0/level" has no "high"         

       schools/0/distance schools/0/level  schools/0/studentsPerTeacher  \
0                     0.1      Elementary                          14.0   
1                     0.4      Elementary                          16.0   
2                     0.3      Elementary                          16.0   
3                     0.2      Elementary                          16.0   
4                     0.3      Elementary                          16.0   
...                   ...             ...                           ...   
75625                 0.2      Elementary                          16.0   
75626                 0.1      Elementary                          16.0   
75627                 0.1      Elementary                          16.0   
75628                 0.4      Elementary                          16.0   
75629                 0.2      Elementary                          16.0   

       schools/1/distance schools/1/level  schools/1/studentsPerTeacher  \
0                     0.

In [10]:
#if we are to drop outliers at point level, we should drop it here.

#Main code
school['s1_High'] = 0 #add a 0 col just for convenience
school['temp_1'] = 0 #need to have these holder cols to use the mean() function so that NaN won't be averaged
school['temp_2'] = 0
school['temp_3'] = 0
stub = ['Elementary','Middle','High']
for i in ['avgDist','avgSpt']:
    stubi = 0
    for j in ['elem','midd','high']:
        col_s = 'school/'+j+'/'+i
        num = 'school/'+j+'/num'
        a = 's1_'+stub[stubi]
        b = 's2_'+stub[stubi]
        c = 's3_'+stub[stubi]
        stubi = stubi + 1
        if i == 'avgDist':
            #if any of the dist or ind is NaN, only that product will be NaN but the mean() will be valid
            school['temp_1'] = school['schools/0/distance']*school[a] #distance*indicator
            school['temp_2'] = school['schools/1/distance']*school[b]
            school['temp_3'] = school['schools/2/distance']*school[c]
            school[col_s] = school[['temp_1','temp_2','temp_3']].mean(axis=1)
        if i == 'avgSpt':
            school['temp_1'] = school['schools/0/studentsPerTeacher']*school[a]
            school['temp_2'] = school['schools/1/studentsPerTeacher']*school[b]
            school['temp_3'] = school['schools/2/studentsPerTeacher']*school[c]
            school[col_s] = school[['temp_1','temp_2','temp_3']].mean(axis=1)
print(school)

       schools/0/distance schools/0/level  schools/0/studentsPerTeacher  \
0                     0.1      Elementary                          14.0   
1                     0.4      Elementary                          16.0   
2                     0.3      Elementary                          16.0   
3                     0.2      Elementary                          16.0   
4                     0.3      Elementary                          16.0   
...                   ...             ...                           ...   
75625                 0.2      Elementary                          16.0   
75626                 0.1      Elementary                          16.0   
75627                 0.1      Elementary                          16.0   
75628                 0.4      Elementary                          16.0   
75629                 0.2      Elementary                          16.0   

       schools/1/distance schools/1/level  schools/1/studentsPerTeacher  \
0                     0.

In [11]:
#Use the folloing code to drop the indicators no longer needed.

#for col in school.columns:
#    for i in ['s1','s2','s3']:
#        if i in col:
#            school = school.drop([col],axis=1)
#print(school)

In [12]:
school.describe()

,schools/0/distance,schools/0/studentsPerTeacher,schools/1/distance,schools/1/studentsPerTeacher,schools/2/distance,schools/2/studentsPerTeacher,s1_Elementary,s1_Middle,s2_Elementary,s2_High,...,s1_High,temp_1,temp_2,temp_3,school/elem/avgDist,school/midd/avgDist,school/high/avgDist,school/elem/avgSpt,school/midd/avgSpt,school/high/avgSpt
count,75499.000000,74504.000000,74719.000000,72815.000000,55543.000000,54985.000000,75630.000000,75630.000000,75630.000000,75630.000000,...,75630.0,74504.0,72815.000000,54985.000000,75499.000000,75499.000000,75499.000000,75090.000000,75090.000000,75090.000000
mean,0.389759,13.921185,0.900305,14.312230,1.166651,17.143894,0.993997,0.004271,0.017586,0.129314,...,0.0,0.0,2.185676,16.847249,0.151732,0.264868,0.354888,5.362341,4.304690,5.235176
std,0.839690,2.048344,1.087598,2.423886,1.244332,2.768755,0.077246,0.065212,0.131440,0.335549,...,0.0,0.0,5.654848,3.659519,0.295931,0.386606,0.410642,1.684159,2.342479,2.557483
min,0.000000,6.000000,0.000000,7.000000,0.000000,7.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.200000,13.000000,0.400000,13.000000,0.500000,15.000000,1.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,15.000000,0.066667,0.100000,0.100000,4.333333,4.000000,4.666667
50%,0.300000,14.000000,0.700000,14.000000,0.900000,18.000000,1.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,18.000000,0.133333,0.200000,0.266667,5.000000,4.666667,6.000000
75%,0.500000,15.000000,1.200000,16.000000,1.600000,19.000000,1.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,19.000000,0.200000,0.350000,0.533333,6.000000,5.333333,6.666667
max,213.100000,20.000000,214.800000,24.000000,208.700000,24.000000,1.000000,1.000000,1.000000,1.000000,...,0.0,0.0,24.000000,24.000000,71.033333,71.600000,69.566667,20.000000,15.500000,20.000000


Problems can be ignored or not:
1. some of the average SPT = 0 
2. outliers

In [13]:
school.columns

Index(['schools/0/distance', 'schools/0/level', 'schools/0/studentsPerTeacher',
       'schools/1/distance', 'schools/1/level', 'schools/1/studentsPerTeacher',
       'schools/2/distance', 'schools/2/level', 'schools/2/studentsPerTeacher',
       's1_Elementary', 's1_Middle', 's2_Elementary', 's2_High', 's2_Middle',
       's3_Elementary', 's3_High', 's3_Middle', 'school/elem/num',
       'school/midd/num', 'school/high/num', 's1_High', 'temp_1', 'temp_2',
       'temp_3', 'school/elem/avgDist', 'school/midd/avgDist',
       'school/high/avgDist', 'school/elem/avgSpt', 'school/midd/avgSpt',
       'school/high/avgSpt'],
      dtype='object')

In [14]:
relevant_columns = ['school/elem/num', 'school/midd/num', 'school/high/num',
                    'school/elem/avgDist', 'school/midd/avgDist', 'school/high/avgDist',
                    'school/elem/avgSpt', 'school/midd/avgSpt', 'school/high/avgSpt']
school_df = school[relevant_columns]

In [15]:
school_df

,school/elem/num,school/midd/num,school/high/num,school/elem/avgDist,school/midd/avgDist,school/high/avgDist,school/elem/avgSpt,school/midd/avgSpt,school/high/avgSpt
0,1,1,0,0.050000,0.050000,0.000000,7.000000,7.0,0.0
1,1,1,0,0.200000,0.150000,0.000000,8.000000,7.5,0.0
2,1,1,0,0.150000,0.200000,0.000000,8.000000,7.5,0.0
3,1,1,0,0.100000,0.150000,0.000000,8.000000,7.5,0.0
4,1,1,0,0.150000,0.200000,0.000000,8.000000,7.5,0.0
...,...,...,...,...,...,...,...,...,...
75625,1,1,1,0.066667,0.200000,0.133333,5.333333,5.0,5.0
75626,1,1,1,0.033333,0.233333,0.333333,5.333333,5.0,5.0
75627,1,1,1,0.033333,0.233333,0.333333,5.333333,5.0,5.0
75628,1,1,1,0.133333,0.300000,0.033333,5.333333,5.0,5.0


In [16]:
excel_file = pd.ExcelFile('census_rank_zip_code.xlsx')
number_of_sheets = len(excel_file.sheet_names)

census_dfs = []
for i in range(number_of_sheets):
    census_dfs.append(pd.read_excel(excel_file, excel_file.sheet_names[i]).iloc[:, 1:])

In [17]:
def get_zipcode_data(df):
    zipcodes = []

    for value in df.iloc[:, -1].values:
        if ' / ' not in str(value):
            zipcodes.append(value)
        else:
            splitted = value.replace(',', '').split(' / ')
            zipcodes.append(splitted[0])

    zipcodes = np.array(zipcodes).astype(int)
    df = df.iloc[:, :-1]
    df['zipcode'] = zipcodes
    return df

In [18]:
for i in range(number_of_sheets):
    census_dfs[i] = get_zipcode_data(census_dfs[i])

In [19]:
def add_zipcode_data(big_df, zipcode_df):
    zipcodes = big_df['address/zipcode'].values
    added_column_name = zipcode_df.columns[0]
    added_values = []

    for i, zipcode in enumerate(zipcodes):
        relevant_row = np.where(zipcode_df['zipcode'].values == zipcode)[0]
        #print(i, zipcode)
        if len(relevant_row) > 0:
            added_values.append(zipcode_df.iloc[relevant_row[0], 0])
        else:
            added_values.append(np.nan)

    #print(len(added_values), big_df.shape)
    big_df[added_column_name] = added_values
    print(added_column_name)
    return big_df

In [20]:
for i in range(number_of_sheets):
    df = add_zipcode_data(df, census_dfs[i])

Average Education Index
Private High School Enrollment Percentage
High School Graduate Education Percentage
Native Born Population Percentage
Foreign Born Population Percentage
White Population Percentage
Black Population Percentage
Asian Population Percentage
Hispanic Population Percentage
Real estate taxes amount
Mortgage interest paid amount
Taxable income amount


In [21]:
df.shape

(75630, 1519)

In [22]:
school_df = pd.concat([school_df, df.iloc[:, -(number_of_sheets):]], axis=1)

In [23]:
school_df.to_csv('census_schools_dataset.csv')

In [24]:
# remove all photo url columns
df = df[df.columns.drop(list(df.filter(regex='photos')))]

# add number of photos attribute
number_of_photos = np.sum(pd.isnull(original_df.filter(like='photos', axis=1).values)==False, axis=1)
df['number_of_photos'] = number_of_photos

df = df.dropna(subset=['price']).reset_index(drop=True)

# remove all listings outside of NY
rows_to_drop = np.where(df['address/state'].values != 'NY')[0]
df = df.drop(df.index[rows_to_drop]).reset_index(drop=True)

#print(df)

In [25]:
#PriceHistory: time format
phist = df.filter(regex = 'priceHistory')
time = [col for col in phist.columns if 'time' in col]
for i in time:
    phist[i]=pd.to_datetime(phist[i],unit='ms')
#print(phist[time])

/tmp/ipykernel_3453282/2083488848.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  phist[i]=pd.to_datetime(phist[i],unit='ms')
/tmp/ipykernel_3453282/2083488848.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  phist[i]=pd.to_datetime(phist[i],unit='ms')
/tmp/ipykernel_3453282/2083488848.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org

In [26]:
event = [col for col in phist.columns if 'event' in col]
#print(phist[event].describe())

In [27]:
#Just printing resoFacts to see what values are contained
rFacts = df.filter(regex = 'resoFactsStats/atAGlanceFacts')
fact = [col for col in rFacts.columns if 'factLabel' in col]
#print(rFacts[fact])